In [ ]:
import os
os.chdir(r"C:\Users\amsas\Desktop\ADLab\rishika_data\Bayleef_2025-01-29_11-58-18_007\Record Node 101\experiment1\recording1\Analysis")
print(os.getcwd())

import matplotlib.pyplot as plt
from pprint import pprint

import spikeinterface as si  # import core only
import spikeinterface.extractors as se
import spikeinterface.preprocessing as spre
import spikeinterface.sorters as ss
import spikeinterface.postprocessing as spost
import spikeinterface.qualitymetrics as sqm
import spikeinterface.comparison as sc
import spikeinterface.exporters as sexp
import spikeinterface.curation as scur
import spikeinterface.widgets as sw
import numpy as np
from probeinterface import ProbeGroup, generate_tetrode



In [ ]:
data_path=r"C:\Users\amsas\Desktop\ADLab\rishika_data\Bayleef_2025-01-29_11-58-18_007\Record Node 101\experiment1\recording1"

recording = se.read_openephys(folder_path=r"C:\Users\amsas\Desktop\ADLab\rishika_data\Bayleef_2025-01-29_11-58-18_007\Record Node 101\experiment1\recording1", stream_id = '0')

In [ ]:
print(recording)
print("Sampling frequency:", recording.get_sampling_frequency())
print("Num channels:", len(recording.get_channel_ids()))
print("Num segments:", recording.get_num_segments())


In [ ]:
# First 1000 timepoints from all channels in the first segment
traces = recording.get_traces(segment_index=0, end_frame=1000)
print(traces.shape)


In [ ]:
print(recording.get_channel_ids())

selection = ['CH'+str(i) for i in np.arange(1,65)]
ephys_recording = recording.select_channels(selection)

print(ephys_recording.get_channel_ids())

In [ ]:
# OLD CHANNEL MAP CODE
# channelmapoe = [40, 38, 36, 34,
#                 48, 46, 44, 42,
#                 56, 54, 52, 50,
#                 58, 64, 62, 60,
#                 63, 61, 59, 57,
#                 55, 53, 51, 49,
#                 47, 45, 43, 41,
#                 39, 37, 35, 33,
#                 25, 27, 29, 31,
#                 17, 19, 21, 23,
#                 9, 11, 13, 15,
#                 1, 3, 5, 7,
#                 4, 6, 8, 2,
#                 10, 12, 14, 16,
#                 18, 20, 22, 24,
#                 26, 28, 30, 32]

# # subtract 1 to fit python indexing
# channelmappythonic = np.array(channelmapoe) - 1

# # create probegroup and set channel locations for sorting purposes
# probegroup = ProbeGroup()
# for i in range(16): # we have 16 tetrodes
#     tetrode = generate_tetrode()
#     tetrode.move([i * 300, 0]) # make lots of space between tetrodes
#     probegroup.add_probe(tetrode)

# probegroup.set_global_device_channel_indices(channelmappythonic) # tetrodes arranged based on channelmap
# ephys_recording = ephys_recording.set_probegroup(probegroup, group_mode='by_probe')

In [ ]:

import ipywidgets as widgets
from IPython.display import display, clear_output

# Channel map (convert to 0-based indexing)
channelmapoe = [40, 38, 36, 34,
                48, 46, 44, 42,
                56, 54, 52, 50,
                58, 64, 62, 60,
                63, 61, 59, 57,
                55, 53, 51, 49,
                47, 45, 43, 41,
                39, 37, 35, 33,
                25, 27, 29, 31,
                17, 19, 21, 23,
                9, 11, 13, 15,
                1, 3, 5, 7,
                4, 6, 8, 2,
                10, 12, 14, 16,
                18, 20, 22, 24,
                26, 28, 30, 32]
channelmappythonic = np.array(channelmapoe) - 1

# Layout parameters
row_tetrode_counts = [2, 6, 6, 2]
x_spacing = 421.64
y_spacing = 365.15
row_shift = 210.82
max_row_width = (max(row_tetrode_counts) - 1) * x_spacing + row_shift * (len(row_tetrode_counts) - 1)

# Compute mirrored positions
original_positions = []
for row_index, count in enumerate(row_tetrode_counts):
    x_offset = row_shift * row_index
    row_width = (count - 1) * x_spacing
    if row_index == 3:
        row_start_x = max_row_width - row_width
    else:
        row_start_x = x_offset
    y = row_index * y_spacing
    row_positions = [(row_start_x + i * x_spacing, y) for i in range(count)]
    original_positions.extend(row_positions)

x_positions = [x for x, _ in original_positions]
x_min, x_max = min(x_positions), max(x_positions)
tetrode_positions = [((x_max - (x - x_min)), y) for x, y in original_positions]

# Create dropdown widgets
tetrode_widgets = []
dropdown_options = [(f"Tetrode {i}", i) for i in range(16)]

for idx in range(16):
    dropdown = widgets.Dropdown(
        options=dropdown_options,
        value=idx,
        description=f"Pos {idx}:",
        layout=widgets.Layout(width='200px')
    )
    tetrode_widgets.append(dropdown)

# Show initial layout and widgets
def plot_layout_with_labels():
    plt.figure(figsize=(10, 6))
    for i, (x, y) in enumerate(tetrode_positions):
        plt.scatter(x, y, s=150)
        plt.text(x, y - 20, f"Pos {i}", ha='center', fontsize=8)
    plt.title("Tetrode Probe Layout (Mirrored)")
    plt.xlabel("X (microns)")
    plt.ylabel("Y (microns)")
    plt.axis('equal')
    plt.grid(True)
    plt.show()

# Display interactive UI
plot_layout_with_labels()
display(widgets.VBox(tetrode_widgets))


In [ ]:
from probeinterface.plotting import plot_probegroup

# Validate and build the ProbeGroup
assigned_tetrodes = [w.value for w in tetrode_widgets]
if len(set(assigned_tetrodes)) != 16:
    print("Error: Each source tetrode (0–15) must be used exactly once.")
else:
    probegroup = ProbeGroup()
    for i, assigned_index in enumerate(assigned_tetrodes):
        tetrode = generate_tetrode()
        tetrode.move(tetrode_positions[i])
        channel_indices = channelmappythonic[assigned_index * 4: (assigned_index + 1) * 4]
        tetrode.set_device_channel_indices(channel_indices)
        tetrode.annotations['assigned_tetrode'] = assigned_index
        probegroup.add_probe(tetrode)

    # Show updated layout and probe group
    clear_output()
    plot_layout_with_labels()
    plot_probegroup(probegroup, same_axes=True)

    print("ProbeGroup built successfully!")
    print("Use with: ephys_recording = ephys_recording.set_probegroup(probegroup, group_mode='by_probe')")

    print("\nFinal Tetrode Assignments:")
    for i, probe in enumerate(probegroup.probes):
        tid = probe.annotations.get("assigned_tetrode", "Unknown")
        chans = list(probe.device_channel_indices)
        print(f"Pos {i} → Tetrode {tid} → Channels: {chans}")
    
    ephys_recording = ephys_recording.set_probegroup(probegroup, group_mode='by_probe')


In [ ]:
from probeinterface.plotting import plot_probegroup

plot_probegroup(probegroup, same_axes=True)

In [ ]:
bad_channel_ids, info = spre.detect_bad_channels(ephys_recording)

ephys_recording = ephys_recording.remove_channels(bad_channel_ids)
print(bad_channel_ids)
pprint(info)
split_recording_dict = ephys_recording.split_by("group")

In [ ]:
print(split_recording_dict.keys())

In [ ]:
for grouper, chan_rec in split_recording_dict.items():
    try:
        print(f"--- Starting group {grouper} ---")

        filtered_recording = spre.bandpass_filter(chan_rec, freq_min=300, freq_max=6000)
        referenced_recording = spre.common_reference(filtered_recording, operator='median')

        sorting = ss.run_sorter(
            sorter_name='mountainsort5',
            recording=referenced_recording,
            folder=f'mountainsort_sorting/group_{grouper}',
            remove_existing_folder=True,
            filter=False,
            verbose=True,
            n_jobs=8
        )

        sorting.save(folder=f'mountainsort_output/group_{grouper}', overwrite=True)
        print(f'Sorted tetrode {grouper}')

        # Skip empty sortings
        if len(sorting.unit_ids) == 0:
            print(f"No units found in group {grouper}. Skipping analyzer.")
            continue

        analyzer = si.create_sorting_analyzer(
            sorting=sorting,
            recording=referenced_recording,
            format="zarr",
            folder=f'analyzer_mountainsort/group_{grouper}',
            overwrite=True
            )
        print('Created analyzer')

        job_kwargs = dict(n_jobs=8, chunk_duration="1s", progress_bar=True)
        compute_dict = {
            'random_spikes': {'method': 'uniform', 'max_spikes_per_unit': 500},
            'waveforms': {'ms_before': 1.0, 'ms_after': 2.0},
            'isi_histograms': {'window_ms': 50.0, 'bin_ms': 1.0, 'method': 'auto'},
            'correlograms': {'window_ms': 50.0, 'bin_ms': 1.0, 'method': 'auto'},
            'templates': {'operators': ["average", "median", "std"]},
            'spike_amplitudes': {},
            'unit_locations': {},
            'template_similarity': {},
            'noise_levels': {},
            'quality_metrics': {}
        }

        analyzer.compute(compute_dict, **job_kwargs)
        print(f'Computed features for group {grouper}')

    except Exception as e:
        print(f"Error in group {grouper}: {e}")


In [ ]:
for grouper, chan_rec in split_recording_dict.items():
    try:
        print(f"--- Starting group {grouper} ---")

        analyzer_path = rf"C:\Users\amsas\Desktop\ADLab\rishika_data\Bayleef_2025-01-29_11-58-18_007\Record Node 101\experiment1\recording1\Analysis\analyzer_mountainsort\group_{group_id}.zarr"

        sorting_analyzer= si.load_sorting_analyzer(analyzer_path)

        analyzer=sorting_analyzer

        job_kwargs = dict(n_jobs=8, chunk_duration="1s", progress_bar=True)
        compute_dict = {
            'quality_metrics': {}
        }

        analyzer.compute(compute_dict, **job_kwargs)
        print(f'Computed features for group {grouper}')

    except Exception as e:
        print(f"Error in group {grouper}: {e}")

In [ ]:
for group_id, chan_rec in split_recording_dict.items():
    print(f"\nGroup {group_id}")
    try:
        traces = chan_rec.get_traces()
        channel_ids = chan_rec.get_channel_ids()
        means = np.mean(traces, axis=0)
        stds = np.std(traces, axis=0)

        for ch_id, mean, std in zip(channel_ids, means, stds):
            print(f"  Channel {ch_id}: mean = {mean:.2f}, std = {std:.2f}")

    except Exception as e:
        print(f"Error reading traces from group {group_id}: {e}")



In [ ]:
for group_id in split_recording_dict.keys():
    print(f"Group {group_id}: {split_recording_dict[group_id].get_channel_ids()}")



In [ ]:
# # # Create an output folder if it doesn't exist
# output_dir = "channel_plots"
# os.makedirs(output_dir, exist_ok=True)

# for group_id, rec in split_recording_dict.items():
#     print(f"Processing group {group_id}...")
#     try:
#         traces = rec.get_traces(start_frame=0, end_frame=15000)  # shape: (timepoints, channels)
#         channel_ids = rec.get_channel_ids()

#         for i, ch_id in enumerate(channel_ids):
#             trace = traces[:, i]
#             plt.figure(figsize=(15, 4))
#             plt.plot(trace, linewidth=0.5)
#             plt.title(f"Group {group_id} - Channel {ch_id}")
#             plt.xlabel("Time (samples)")
#             plt.ylabel("Amplitude")
#             plt.tight_layout()

#             filename = f"group{group_id}_channel{ch_id}.png"
#             filepath = os.path.join(output_dir, filename)
#             plt.savefig(filepath, dpi=150)
#             plt.close()

#     except Exception as e:
#         print(f"Error in group {group_id}: {e}")

In [ ]:
# #spikeinterface curation gui

# sorting_analyzer = si.load_sorting_analyzer(r"C:\Users\amsas\Desktop\ADLab\rishika_data\Bayleef_2025-01-29_11-58-18_007\Record Node 101\experiment1\recording1\Analysis\analyzer_mountainsort\group_9.zarr")
# sw.plot_sorting_summary(sorting_analyzer, backend="spikeinterface_gui", curation=True)

In [ ]:
for group_id in range(16):
    try:
        analyzer_path = rf"C:\Users\amsas\Desktop\ADLab\rishika_data\Bayleef_2025-01-29_11-58-18_007\Record Node 101\experiment1\recording1\Analysis\analyzer_mountainsort\group_{group_id}.zarr"

        sorting_analyzer= si.load_sorting_analyzer(analyzer_path)

        print(sorting_analyzer.get_loaded_extension_names())

        metrics = sqm.compute_quality_metrics(sorting_analyzer, metric_names=[
            "snr",
            "isi_violation",
            "nearest_neighbor",
            "firing_rate",
            "isi_violation",
            "l_ratio",
            "isolation_distance",
            "presence_ratio"
            ])
        
        print(f"Metrics for group_{group_id}")
        print(metrics)

    except Exception as e:
        print(f"Error in group {group_id}: {e}")


In [ ]:
from spikeinterface.exporters import export_to_phy

for group_id in range(16):
    try:
        analyzer_path = rf"C:\Users\amsas\Desktop\ADLab\rishika_data\Bayleef_2025-01-29_11-58-18_007\Record Node 101\experiment1\recording1\Analysis\analyzer_mountainsort\group_{group_id}.zarr"
        output_path = rf"C:\Users\amsas\Desktop\ADLab\rishika_data\Bayleef_2025-01-29_11-58-18_007\Record Node 101\experiment1\recording1\Analysis\phy_output\group_{group_id}_phy"

        if not os.path.exists(analyzer_path):
            print(f"Skipping group {group_id} — no analyzer found.")
            continue

        print(f"Exporting group {group_id} to Phy...")

        sorting_analyzer = si.load_sorting_analyzer(analyzer_path)

        export_to_phy(
            sorting_analyzer=sorting_analyzer,
            output_folder=output_path,
            compute_pc_features=True,
            compute_amplitudes=True,
        )

        print(f"Exported to Phy for group {group_id}")

    except Exception as e:
        print(f"Error exporting group {group_id}: {e}")


In [ ]:
all_keep_units = {}

for group_id in range(16):
    try:
        analyzer_path = rf"C:\Users\amsas\Desktop\ADLab\rishika_data\Bayleef_2025-01-29_11-58-18_007\Record Node 101\experiment1\recording1\Analysis\analyzer_mountainsort\group_{group_id}.zarr"
        if not os.path.exists(analyzer_path):
            print(f"Skipping group {group_id} — analyzer not found.")
            continue

        print(f"\nEvaluating group {group_id}...")

        sorting_analyzer = si.load_sorting_analyzer(analyzer_path)

        # Compute noise_levels if missing
        if not sorting_analyzer.has_extension("noise_levels"):
            print("Computing noise_levels...")
            sorting_analyzer.compute("noise_levels")

        # Then compute quality metrics
        if not sorting_analyzer.has_extension("quality_metrics"):
            print("Computing quality_metrics...")
            sorting_analyzer.compute("quality_metrics")

        # Retrieve metrics
        metrics = sorting_analyzer.get_extension("quality_metrics").get_data()

        # Apply your thresholds
        keep_mask = (
            (metrics["snr"] > 7.5) &
            (metrics["isi_violations_ratio"] < 0.2) &
            (metrics["nn_hit_rate"] > 0.90)
        )

        keep_unit_ids = keep_mask[keep_mask].index.to_list()
        total_units = len(metrics)
        good_units = len(keep_unit_ids)

        print(f"Total units in group {group_id}: {total_units}")
        print(f"Good units in group {group_id}: {good_units} -> {keep_unit_ids}")

        all_keep_units[group_id] = keep_unit_ids

    except Exception as e:
        print(f"Error processing group {group_id}: {e}")


In [ ]:
for group_id in range(16):
    try:
        analyzer_path = rf"C:\Users\amsas\Desktop\ADLab\rishika_data\Bayleef_2025-01-29_11-58-18_007\Record Node 101\experiment1\recording1\Analysis\analyzer_mountainsort\group_{group_id}.zarr"
        if not os.path.exists(analyzer_path):
            print(f"Skipping group {group_id} — analyzer not found.")
            continue

        print(f"Plotting group {group_id}...")
        sorting_analyzer = si.load_sorting_analyzer(analyzer_path)
        unit_ids = sorting_analyzer.sorting.unit_ids

        if len(unit_ids) == 0:
            print(f"Group {group_id} has no units. Skipping plots.")
            continue

        # Labeling each figure with the group ID
        def label_figure(title):
            plt.suptitle(f"Group {group_id} - {title}", fontsize=16)

        sw.plot_unit_waveforms(sorting_analyzer, unit_ids=unit_ids, figsize=(16, 4))
        label_figure("Unit Waveforms")

        sw.plot_unit_templates(sorting_analyzer, unit_ids=unit_ids, ncols=5, figsize=(16,8))
        label_figure("Unit Templates")

        sw.plot_unit_locations(sorting_analyzer, unit_ids=unit_ids)
        label_figure("Unit Locations")

        sw.plot_amplitudes(sorting_analyzer, plot_histograms=True, figsize=(12,8))
        label_figure("Amplitudes")

        plt.show()  # Show all plots before next group

    except Exception as e:
        print(f"Error in group {group_id}: {e}")
